In [1]:
import numpy as np
import torch
from torchvision import models
from sklearn.model_selection import StratifiedShuffleSplit
from data_utils import *
%load_ext autoreload
%autoreload 2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [2]:
## Load the data
data = load_bolete_data()

ItemsViewHDF5(<HDF5 file "bolete.h5" (mode r)>)


In [6]:
# Get Train and Test Split
X_train, X_test, Y_train, Y_test = get_train_and_test(data, 'bolete-characteristics')


In [14]:
## Download a pretrained model
googlenet = models.googlenet(pretrained=True)
rex101 = models.resnext101_32x8d(pretrained=True)
alexnet = models.alexnet(pretrained=True)

In [15]:
## Transfer Learn on the new data with cross validation
## inspired by https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

import torch.optim as optim
import torch.nn as nn
from train import cross_val
import torchvision.transforms as T

dtype = torch.cuda.FloatTensor if use_cuda else torch.float32

def loss_fn(scores, true_y):
    print(sum(scores == true_y))
    return sum(scores == true_y)

def pred_fn(scores):
    outputs = torch.sigmoid(scores)
    preds = outputs > 0.5
    return preds

num_classes = Y_train.shape[1]

googlenet.fc = nn.Linear(1024, num_classes)


alexnet.classifier[6] = nn.Linear(4096, num_classes)

optimizer = optim.Adam(googlenet.parameters(), betas=(.5, .999), lr=1e-3)
model = alexnet

# print(model)

transform = T.Compose([T.ToTensor()])

cross_val(
    X_train, 
    Y_train, 
    model, 
    optimizer, 
    loss_fn, 
    pred_fn,
    64, 
    10,
    5,
    device,
    transform = T.Compose([T.ToTensor()]))

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn